In [1]:
!pip install transformers==4.5.0 fugashi==1.1.0 ipadic==1.0.0

     |████████████████████████████████| 2.1 MB 3.2 MB/s 
     |████████████████████████████████| 486 kB 9.8 MB/s 
     |████████████████████████████████| 13.4 MB 20.7 MB/s 
     |████████████████████████████████| 895 kB 19.4 MB/s 
     |████████████████████████████████| 3.3 MB 38.8 MB/s 
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556723 sha256=2c0ecf54097fa50ade9630162451123f0fea439400318576f06bf07dd3ec28d7
  Stored in directory: /root/.cache/pip/wheels/33/8b/99/cf0d27191876637cd3639a560f93aa982d7855ce826c94348b
Successfully built ipadic


In [2]:
import numpy as np
import torch
from transformers import BertJapaneseTokenizer, BertForMaskedLM

In [3]:
model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)
bert_mlm = BertForMaskedLM.from_pretrained(model_name)
bert_mlm = bert_mlm.cuda()

Downloading:   0%|          | 0.00/258k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/110 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/479 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [38]:
text = '今日は[MASK]へ行く。'
tokens = tokenizer.tokenize(text)
print(tokens)

['今日', 'は', '[MASK]', 'へ', '行く', '。']


In [39]:
# 文章を符号化し、GPUに配置する。
input_ids = tokenizer.encode(text, return_tensors='pt')
input_ids = input_ids.cuda()

# BERTに入力し、分類スコアを得る。
# 系列長を揃える必要がないので、単にiput_idsのみを入力します。
with torch.no_grad():
    output = bert_mlm(input_ids=input_ids)
    scores = output.logits

In [40]:
mask_position = input_ids[0].tolist().index(4) 
print(scores[0, mask_position].shape)
print(scores[0, mask_position])
print(scores[0, mask_position].argmax(-1))
print(scores[0, mask_position].argsort()[-5:])

torch.Size([32000])
tensor([-5.4308,  5.4983, -4.1217,  ..., -3.7345, -5.4048, -2.9598],
       device='cuda:0')
tensor(391, device='cuda:0')
tensor([5359, 1724,  466, 6166,  391], device='cuda:0')


In [50]:
# ID列で'[MASK]'(IDは4)の位置を調べる
mask_position = input_ids[0].tolist().index(4) 

id_best = scores[0, mask_position].argmax(-1).item()
print(id_best)

# スコアが良い方から N 個のトークンのIDを取り出し、トークンに変換する。
n = 10
ids = scores[0, mask_position].argsort()[-n:]
print(ids)
print(tokenizer.convert_ids_to_tokens(6166))

for i, id in enumerate(reversed(ids)):
    id = id.item()
    print(id)
    token_by_id = tokenizer.convert_ids_to_tokens(id)
    token_by_id = token_by_id.replace('##', '')

    # [MASK]を上で求めたトークンで置き換える。
    print(f'===== TOP{i+1:2} =====')
    predicted_text = text.replace('[MASK]', token_by_id)

    print(predicted_text)

391
tensor([2249, 1221, 2030,  286, 2118, 5359, 1724,  466, 6166,  391],
       device='cuda:0')
ハワイ
391
===== TOP 1 =====
今日は東京へ行く。
6166
===== TOP 2 =====
今日はハワイへ行く。
466
===== TOP 3 =====
今日は学校へ行く。
1724
===== TOP 4 =====
今日はニューヨークへ行く。
5359
===== TOP 5 =====
今日はどこへ行く。
2118
===== TOP 6 =====
今日は空港へ行く。
286
===== TOP 7 =====
今日はアメリカへ行く。
2030
===== TOP 8 =====
今日は病院へ行く。
1221
===== TOP 9 =====
今日はそこへ行く。
2249
===== TOP10 =====
今日はロンドンへ行く。


In [51]:
def predict_mask_topk(text, tokenizer, bert_mlm, num_topk):
    """
    文章中の最初の[MASK]をスコアの上位のトークンに置き換える。
    上位何位まで使うかは、num_topkで指定。
    出力は穴埋めされた文章のリストと、置き換えられたトークンのスコアのリスト。
    """
    # 文章を符号化し、BERTで分類スコアを得る。
    input_ids = tokenizer.encode(text, return_tensors='pt')
    input_ids = input_ids.cuda()
    with torch.no_grad():
        output = bert_mlm(input_ids=input_ids)
    scores = output.logits

    # スコアが上位のトークンとスコアを求める。
    mask_position = input_ids[0].tolist().index(4) 
    topk = scores[0, mask_position].topk(num_topk)
    ids_topk = topk.indices # トークンのID
    tokens_topk = tokenizer.convert_ids_to_tokens(ids_topk) # トークン
    scores_topk = topk.values.cpu().numpy() # スコア

    # 文章中の[MASK]を上で求めたトークンで置き換える。
    text_topk = [] # 穴埋めされたテキストを追加する。
    for token in tokens_topk:
        token = token.replace('##', '')
        text_topk.append(text.replace('[MASK]', token, 1))

    return text_topk, scores_topk

text = '今日は[MASK]へ行く。'
text_topk, _ = predict_mask_topk(text, tokenizer, bert_mlm, 10)
print(*text_topk, sep='\n')

今日は東京へ行く。
今日はハワイへ行く。
今日は学校へ行く。
今日はニューヨークへ行く。
今日はどこへ行く。
今日は空港へ行く。
今日はアメリカへ行く。
今日は病院へ行く。
今日はそこへ行く。
今日はロンドンへ行く。


In [56]:
def greedy_prediction(text, tokenizer, bert_mlm):
    """
    [MASK]を含む文章を入力として、貪欲法で穴埋めを行った文章を出力する。
    """
    # 前から順に[MASK]を一つづつ、スコアの最も高いトークンに置き換える。
    for _ in range(text.count('[MASK]')):
        print(text)
        text = predict_mask_topk(text, tokenizer, bert_mlm, 1)[0][0]
    return text

# text = '今日は[MASK][MASK]へ行く。'
text = '今日は[MASK][MASK][MASK]行く。'
greedy_prediction(text, tokenizer, bert_mlm)

今日は[MASK][MASK][MASK]行く。
今日は、[MASK][MASK]行く。
今日は、東京[MASK]行く。


'今日は、東京へ行く。'

In [57]:
# このようなタスクはあまり得意ではない。
# GPT とかは現在までのトークンから次のトークンを予測するタスクを取っているのでこういうこともできる！
text = '今日は[MASK][MASK][MASK][MASK][MASK]'
greedy_prediction(text, tokenizer, bert_mlm)

今日は[MASK][MASK][MASK][MASK][MASK]
今日は社会[MASK][MASK][MASK][MASK]
今日は社会社会[MASK][MASK][MASK]
今日は社会社会的[MASK][MASK]
今日は社会社会的な[MASK]


'今日は社会社会的な地位'

In [58]:
def beam_search(text, tokenizer, bert_mlm, num_topk):
    """
    ビームサーチで文章の穴埋めを行う。
    """
    num_mask = text.count('[MASK]')
    text_topk = [text]
    scores_topk = np.array([0])
    for _ in range(num_mask):
        # 現在得られている、それぞれの文章に対して、
        # 最初の[MASK]をスコアが上位のトークンで穴埋めする。
        text_candidates = [] # それぞれの文章を穴埋めした結果を追加する。
        score_candidates = [] # 穴埋めに使ったトークンのスコアを追加する。
        for text_mask, score in zip(text_topk, scores_topk):
            text_topk_inner, scores_topk_inner = predict_mask_topk(
                text_mask, tokenizer, bert_mlm, num_topk
            )
            text_candidates.extend(text_topk_inner)
            score_candidates.append( score + scores_topk_inner )

        # 穴埋めにより生成された文章の中から合計スコアの高いものを選ぶ。
        score_candidates = np.hstack(score_candidates)
        idx_list = score_candidates.argsort()[::-1][:num_topk]
        text_topk = [ text_candidates[idx] for idx in idx_list ]
        scores_topk = score_candidates[idx_list]

    return text_topk

text = "今日は[MASK][MASK]へ行く。"
text_topk = beam_search(text, tokenizer, bert_mlm, 10)
print(*text_topk, sep='\n')

今日はお台場へ行く。
今日はお祭りへ行く。
今日はゲームセンターへ行く。
今日はお風呂へ行く。
今日はゲームショップへ行く。
今日は東京ディズニーランドへ行く。
今日はお店へ行く。
今日は同じ場所へ行く。
今日はあの場所へ行く。
今日は同じ学校へ行く。


In [59]:
text = '今日は[MASK][MASK][MASK][MASK][MASK]'
text_topk = beam_search(text, tokenizer, bert_mlm, 10)
print(*text_topk, sep='\n')

今日は社会社会学会所属。
今日は社会社会学会会長。
今日は社会社会に属する。
今日は時代社会に属する。
今日は社会社会学会理事。
今日は時代社会にあたる。
今日は社会社会にある。
今日は社会社会学会会員。
今日は時代社会にある。
今日は社会社会になる。
